In [ ]:
# 03_momentum_strategy.ipynb

# Importing necessary libraries
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Downloading sample stock data (e.g., Apple stock)
data = yf.download('AAPL', start='2024-01-01', end='2025-01-01')

# Calculate momentum using the relative strength index (RSI)
def compute_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Adding the RSI to the data
data['RSI'] = compute_rsi(data)

# Defining the strategy: Buy if RSI < 30 (oversold), sell if RSI > 70 (overbought)
data['Signal'] = np.where(data['RSI'] < 30, 1, np.where(data['RSI'] > 70, -1, 0))

# Simulating a simple trading strategy
data['Daily_Return'] = data['Close'].pct_change()
data['Strategy_Return'] = data['Signal'].shift(1) * data['Daily_Return']  # Strategy uses previous day's signal

# Calculate the cumulative returns of the strategy
data['Cumulative_Return'] = (1 + data['Strategy_Return']).cumprod()

# Plotting the strategy's cumulative return vs stock price
plt.figure(figsize=(12,6))
plt.plot(data['Cumulative_Return'], label='Strategy Cumulative Return')
plt.plot((1 + data['Daily_Return']).cumprod(), label='Stock Cumulative Return', alpha=0.7)
plt.title('Momentum Strategy vs Stock Performance')
plt.legend(loc='best')
plt.show()
